In [1]:
import tensorflow as tf 
import tensorflow_hub as hub 
from tensorflow.keras import layers 
import bert 
import re 
# re — Regular expression operations
import math
import csv
import pandas as pd                     
import cv2 as cv 
from PIL import Image, ImageSequence
from tensorflow.keras import losses
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import sys

In [2]:
data = pd.read_csv("/Users/zimingfang/Desktop/Animated GIFs/AwesomeGif/tgif-v1.0.tsv", sep='\t')
data.isnull().values.any()
data.shape

(125782, 2)

In [3]:
data.head()

,y,x
0,https://38.media.tumblr.com/9f6c25cc350f12aa74...,"a man is glaring, and someone with sunglasses ..."
1,https://38.media.tumblr.com/9ead028ef62004ef6a...,a cat tries to catch a mouse on a tablet
2,https://38.media.tumblr.com/9f43dc410be85b1159...,a man dressed in red is dancing.
3,https://38.media.tumblr.com/9f659499c8754e40cf...,an animal comes close to another in the jungle
4,https://38.media.tumblr.com/9ed1c99afa7d714118...,a man in a hat adjusts his tie and makes a wei...


In [4]:
gif_links = list(data.y.values)
raw_tweets = list(data.x.values)

In [5]:
# gif_links

In [6]:
# raw_tweets

# Tweet Pre-Process 

## Remove Special Char

In [7]:
# definition for function for removing html tags 
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [8]:
# definition for function for remove any punctuations and special characters
def preprocess_text(raw_tweaet):
    # Removing html tags
    tweet = remove_tags(raw_tweaet)
    # Removing html tags
    tweet = re.sub('[^a-zA-Z]', '', tweet)
    # Removing html tags
    tweet = re.sub(r"\s+[a-zA-Z]\s+", ' ', tweet)
    # Removing multiple spaces
    tweet = re.sub(r'\s+', ' ', tweet)
    return tweet

In [9]:
# run the preprocess_text function to clean tweets list 
tweets = [] 
for tweet in raw_tweets[:50]:
    tweets.append(preprocess_text(tweet))

## Tokenizing 

In [10]:
# Create a tokenizer 
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
# .numpy(): converts a tensor object into an numpy.ndarray
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

KeyboardInterrupt: 

In [ ]:
# Definition for function for convert tweet to ids 
def tokenize_tweets(text_tweets):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_tweets))

In [ ]:
# run the tokenize_tweets on tweets 
tokenized_tweets = [tokenize_tweets(tweet) for tweet in tweets]

# GIF Pre-Process

In [11]:
import requests
# Requests is an elegant and simple HTTP library for Python
import os 
# os — Miscellaneous operating system interfaces¶
os.chdir('/Users/zimingfang/Desktop/Animated GIFs/AwesomeGif/gifs')
# !pwd

In [12]:
def gif_downloader(image_urls, status=[], filenames = []):
    
    for index, img in enumerate(image_urls):
        # We can split the file based upon / and extract the last split within the python list below:
        file_name = img.split('/')[-1]
        #print("fThis is the file name: {file_name}")
        filenames.append(file_name) 
        # Now let's send a request to the image URL:
        r = requests.get(img, stream=True)
        # We can check that the status code is 200 before doing anything else:
        if r.status_code == 200:
            # This command below will allow us to write the data to a file as binary:
            with open(file_name, 'wb') as f:
                for chunk in r:
                    f.write(chunk)
            status.append(True)
        else:
            # We will write all of the images back to the broken_images list:
            status.append(False)
    return filenames, status

In [13]:
# def load_gif(image_path, sess):
#     image = tf.io.read_file(image_file)
#     image = tf.io.decode_gif(image)
#     return sess.run(image)

In [14]:
download_status = []
filenames = []
filenames, downlod_status = gif_downloader(gif_links[:50], status=download_status)
# if any gif was not downloaded successfully 
print(any(status == False for status in download_status))

False


In [15]:
# filenames

In [22]:
def calculate_gif_entropy(path_to_gif_file):
    capture = Image.open(path_to_gif_file)
    gif_entropy = []
    for frame in ImageSequence.Iterator(capture):
        gif_entropy.append(frame.entropy())
    return sum(gif_entropy) / len(gif_entropy)
#     return len(brightness)

In [23]:
# In GIF files, each frame has its own duration. So there is no general fps for a GIF file. 
entropy = []
for gif_file_path in filenames[:50]: 
    entropy.append(calculate_gif_entropy(gif_file_path))

In [24]:
# entropy

[5.170089872944317,
 5.225727671716859,
 5.840859871407656,
 3.6871331153782667,
 3.205473205169353,
 5.121113802108533,
 6.214433791065296,
 6.7225877440813955,
 6.339215119628434,
 5.170476977981715,
 6.070336507125528,
 3.2055572805088093,
 4.73859891579478,
 6.561740153258861,
 4.909666446255826,
 5.904178349492388,
 6.103436802190598,
 4.963691713804072,
 4.89823756025263,
 3.8382394307371377,
 3.9135805499622487,
 5.445038184743639,
 3.0329605071907455,
 3.7634516982135384,
 6.602547717816006,
 5.152454991847544,
 4.499061360781062,
 5.234744028076599,
 6.530786806777977,
 4.6212382337793185,
 5.1204861189334085,
 4.082479347046767,
 3.280546318276212,
 4.195160122803214,
 5.498736657632448,
 6.762598954892426,
 4.816196415332943,
 5.224755910511213,
 3.8860765762415235,
 5.596349431448027,
 5.690443725371455,
 6.024895909471839,
 3.231666145215634,
 5.190883520201109,
 3.992409043522016,
 6.659291979605626,
 5.293323805824246,
 3.604181159146925,
 3.2852856112720032,
 3.28528561

# Prerparing Data For Training


In [ ]:
x_train = tokenized_tweets
y_train = entropy

In [ ]:
# len(x_train)

In [ ]:
tweets_with_len = [[tweet, y_train[i], len(tweet)]
                 for i, tweet in enumerate(x_train)]

In [ ]:
#sort the data by tweet length 
tweets_with_len.sort(key=lambda x: x[2])
#remove the tweet length attribute from dataset 
sorted_tweets_labels = [(tweet_lab[0], tweet_lab[1]) for tweet_lab in tweets_with_len]

In [ ]:
# sorted_tweets_labels

In [ ]:
# convert the sorted dataset into a TensorFlow 2.0-compliant input dataset shape
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_tweets_labels, output_types=(tf.int32, tf.float32))


In [ ]:
BATCH_SIZE = 10 
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
TOTAL_BATCHES = math.ceil(len(sorted_tweets_labels) / BATCH_SIZE)
# TEST_BATCHES = TOTAL_BATCHES // 10
TEST_BATCHES = 1
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [ ]:
# next(iter(batched_dataset))

# Creating the Model


In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200

In [ ]:
#GlobalAveragePooling1D layer returns a fixed-length output vector for each example by averaging over the sequence dimension.
#allows the model to handle input of variable length, in the simplest way possible

model = tf.keras.Sequential([
    layers.Embedding(VOCAB_LENGTH + 1, EMB_DIM),
    layers.Dropout(0.2),    
    layers.GlobalAveragePooling1D(), 
    layers.Dropout(0.2),
    layers.Dense(1, activation='linear') #set activation to linear for regression
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

In [ ]:
epochs = 100
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=epochs)

In [ ]:
print(history.history.keys())

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=history.history['loss'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=history.history['val_loss'],
                     name='Valid'))

fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()

In [ ]:
results = model.evaluate(test_data)

print(results)